In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, tensorflow as tf, cv2, shutil
from tqdm import tqdm
tqdm.pandas()
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import save_img
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, BatchNormalization, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.callbacks import TensorBoard
from glob import glob
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.efficientnet import EfficientNetB2, EfficientNetB7

In [2]:
# shutil.rmtree("/kaggle/working/")   ## To delete all directories present in output

In [3]:
test_csv = pd.read_csv('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/test.csv')
test_csv.shape

(6636, 1)

In [4]:
train_csv = pd.read_csv('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/train.csv')
train_csv.shape

(19906, 2)

In [5]:
# train_csv['Class'].unique()

In [6]:
## To see distribution of different classes
dist = pd.DataFrame(train_csv['Class'].value_counts()/train_csv.shape[0]*100)
# dist.rename(columns = {'Class': 'Percentage'}, inplace = True) ## To replace the columns name 
dist.columns = ['Percentage'] ## To replace the columns name if there is only one column
dist.index = ['MIDDLE', 'OLD', 'YOUNG']
dist.sort_values(by= ['Percentage'], ascending = False)
dist

,Percentage
MIDDLE,54.275093
OLD,33.688335
YOUNG,12.036572


In [7]:
# train_csv['Class'].value_counts(dropna = False).plot(kind = 'bar',grid = True)
# plt.title("Distribtuion of class counts in the training set")
# plt.xticks(rotation = 0)

In [8]:
# train_csv.head()

In [9]:
## Label Encoding
label_encoder = preprocessing.LabelEncoder()
train_csv['Class'] = label_encoder.fit_transform(train_csv['Class'])
train_csv['Class'].unique()

array([0, 2, 1])

In [10]:
# train_csv.head()

**Hence, following encoding took place: Middle--> 0, Old--> 1, Young--> 2 (In alphabetical order)**

In [11]:
## Training and validation set spliting
# train_csv_valid = train_csv[:6569] ## 0.33 valid set
# train_csv_train = train_csv[6570:] ## 0.66 train set
# train_csv_valid = train_csv.sample(frac=0.33)
# train_csv_train = train_csv.drop(train_csv_valid.index)
# print(train_csv_valid.shape)
# print(train_csv_train.shape)

## Train validation split using Stratified K-fold cross validation technique
# X = train_csv['ID']
# y = train_csv['Class']
# skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
# for train_idx, valid_idx in list(skf.split(X,y)):
#     train_csv_train = train_csv.iloc[train_idx]
#     train_csv_valid = train_csv.iloc[valid_idx]
# #     print(train_csv_train.head(10))

In [12]:
# X = train_csv['ID']
# X.shape

In [13]:
# train_csv_train.head()

In [14]:
!mkdir /kaggle/working/Image_set/

!mkdir /kaggle/working/Image_set/Train
!mkdir /kaggle/working/Image_set/Train/0
!mkdir /kaggle/working/Image_set/Train/1
!mkdir /kaggle/working/Image_set/Train/2

!mkdir /kaggle/working/Image_set/Valid
!mkdir /kaggle/working/Image_set/Valid/0
!mkdir /kaggle/working/Image_set/Valid/1
!mkdir /kaggle/working/Image_set/Valid/2

!mkdir /kaggle/working/Image_set/Test
!mkdir /kaggle/working/Image_set/Test/test

## Middle- 0, Old- 1, Young- 2

### Resizing and Splitting the image dataset into Valid and Train set

In [15]:
# ## for training dataset
# for index, row in tqdm(train_csv_train.iterrows(),total=len(train_csv_train)):
#     image = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
#     image = cv2.resize(image,(224,224))
#     img_path = "/kaggle/working/Train/"+str(row['Class'])+"/"+row['ID']
#     save_img(img_path,image)

In [16]:
# ## for validation dataset
# for index, row in tqdm(train_csv_valid.iterrows(),total=len(train_csv_valid)):
#     image = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
#     image = cv2.resize(image,(224,224))
#     img_path = "/kaggle/working/Valid/"+str(row['Class'])+"/"+row['ID']
#     save_img(img_path,image)

In [17]:
# train_csv_valid.head()

### Resizing the test image dataset

In [18]:
# test_csv.head()

In [19]:
# for index, row in tqdm(test_csv.iterrows(),total=len(test_csv)):
#     image = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/Test/'+row['ID'])
#     image = cv2.resize(image,(224,224))
#     img_path = "/kaggle/working/Test/test/"+row['ID']
#     save_img(img_path,image)

In [20]:
# import os
# os.listdir("/kaggle/working/Test")

In [21]:
# from PIL import Image
# image1 = Image.open('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/Test/10.jpg')
# # plt.imshow(image1)
# image1.size

In [22]:
# for index, row in tqdm(test_csv.iterrows(), total = len(test.csv)):
#     shutil.copy2('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/Test/'+row['ID'], '/kaggle/working/Test/test/')

In [23]:
# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/',
#                                   target_size = (224,224),
#                                   batch_size = 32,
#                                   class_mode = 'categorical',
#                                     color_mode='rgb')

In [24]:
# test_set[0][0][0].shape[0]

In [25]:
# test_csv.shape[0]

In [26]:
# test_set[1][0][32].shape

In [27]:
# j = 0
# for i in range(test_csv.shape[0]):
#     j += 1
#     if(test_set[j][0][i].shape[0] != 64):
#         print(test_set[0][0][i].shape[0])
# # print('Program ended!')
# # # test_set[0][0][0].shape

In [28]:
# batch_size=32
# # To handle image loading problem
# from PIL import Image, ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# X = train_csv['ID']
# y = train_csv['Class']
# skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
# i = 1
# for train_idx, valid_idx in list(skf.split(X,y)):
#     if (i!=1):
#         !mkdir /kaggle/working/Image_set/

#         !mkdir /kaggle/working/Image_set/Train
#         !mkdir /kaggle/working/Image_set/Train/0
#         !mkdir /kaggle/working/Image_set/Train/1
#         !mkdir /kaggle/working/Image_set/Train/2

#         !mkdir /kaggle/working/Image_set/Valid
#         !mkdir /kaggle/working/Image_set/Valid/0
#         !mkdir /kaggle/working/Image_set/Valid/1
#         !mkdir /kaggle/working/Image_set/Valid/2

#     train_csv_train = train_csv.iloc[train_idx]
#     train_csv_valid = train_csv.iloc[valid_idx]
#     ## for training dataset
#     for index, row in tqdm(train_csv_train.iterrows(),total=len(train_csv_train)):
#         image_train = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
#         image_train = cv2.resize(image_train,(224,224))
#         img_path_train = "/kaggle/working/Image_set/Train/"+str(row['Class'])+"/"+row['ID']
#         save_img(img_path_train,image_train)
#     ## for validation dataset
#     for index, row in tqdm(train_csv_valid.iterrows(),total=len(train_csv_valid)):
#         image_valid = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
#         image_valid = cv2.resize(image_valid,(224,224))
#         img_path_valid = '/kaggle/working/Image_set/Valid/'+str(row['Class'])+"/"+row['ID']
#         save_img(img_path_valid,image_valid)
        
# #      ------------------------TRAINING AND VALIDATION SET------------------------
#     train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True, 
#                                   rotation_range = 10, fill_mode = 'nearest',
#                                   width_shift_range = 0.2, height_shift_range=0.2)
#     val_datagen = ImageDataGenerator(rescale = 1./255)
#     training_set = train_datagen.flow_from_directory('/kaggle/working/Image_set/Train/',
#                                    target_size = (224,224),
#                                    batch_size = batch_size,
#                                    class_mode = 'categorical',
#                                     color_mode='rgb')
#     validation_set = val_datagen.flow_from_directory('/kaggle/working/Image_set/Valid/',
#                                   target_size = (224,224),
#                                   batch_size = batch_size,
#                                   class_mode = 'categorical',
#                                     color_mode='rgb')
    
# #     ------------------------TRAINING THE MODELS------------------------
#     FC_LAYERS = [1024, 512, 256]
#     dropout = 0.5
#     def build_model(base_model, dropout, fc_layers, num_classes):
#         for layer in base_model.layers:
#             layer.trainable = True

#         x = base_model.output
#         x = Flatten()(x)
#         for fc in fc_layers:
#             print(fc)
#             x = Dense(fc, activation='relu')(x)
#             x = Dropout(dropout)(x)
#         preditions = Dense(num_classes, activation='softmax')(x)
#         finetune_model = Model(inputs = base_model.input, outputs = preditions)
#         return finetune_model

#     base_model_1 = ResNet50(weights = 'imagenet',
#                            include_top = False,
#                            input_shape = (224,224,3))

#     base_model_2 = InceptionV3(weights = 'imagenet',
#                            include_top = False,
#                            input_shape = (224,224, 3))

#     base_model_3 = DenseNet121(weights = 'imagenet',
#                            include_top = False,
#                            input_shape = (224,224, 3))

#     base_model_4 = EfficientNetB7(include_top= False,
#                                   weights="imagenet",
#                                   input_shape=(224,224,3))
    
# #     -----------------------RSNT MODEL-----------------------
#     # Training model
#     Rsnt_model = build_model(base_model_1,
#                             dropout = dropout,
#                             fc_layers = FC_LAYERS,
#                             num_classes = 3)

#     # compiling the model
#     opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.0001,decay=1e-4)
#     Rsnt_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
#     # fitting the model
#     history_rsnt = Rsnt_model.fit_generator(training_set,
#                              steps_per_epoch = len(training_set),
#                              epochs = 10,
#                              validation_data = validation_set,
#                              validation_steps = len(validation_set),
#                              verbose = 1)

#     # Saving model
#     Rsnt_model.save('/kaggle/working/Rsnt_'+str(i))
    
# #     -----------------------INCEPTION MODEL-----------------------
#     # Training model
#     Inception_model = build_model(base_model_2,
#                             dropout = dropout,
#                             fc_layers = FC_LAYERS,
#                             num_classes = 3)

#     # compiling the model
#     opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.0001,decay=1e-4)
#     Inception_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
#     # fitting the model
#     history_inception = Inception_model.fit_generator(training_set,
#                              steps_per_epoch = len(training_set),
#                              epochs = 10,
#                              validation_data = validation_set,
#                              validation_steps = len(validation_set),
#                              verbose = 1)

#     # Saving model
#     Inception_model.save('/kaggle/working/Inception_'+str(i))
    
# # -----------------------DENSNET MODEL-----------------------
#     # Training model
#     Densenet_model = build_model(base_model_3,
#                             dropout = dropout,
#                             fc_layers = FC_LAYERS,
#                             num_classes = 3)

#     # compiling the model
#     opt = tf.keras.optimizers.legacy.RMSprop(learning_rate = 0.0001,decay=1e-4)
#     Densenet_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
#     # fitting the model
#     history_densenet = Densenet_model.fit_generator(training_set,
#                              steps_per_epoch = len(training_set),
#                              epochs = 10,
#                              validation_data = validation_set,
#                              validation_steps = len(validation_set),
#                              verbose = 1)

#     # Saving model
#     Densenet_model.save('/kaggle/working/Densenet'+str(i))
    
# # -----------------------EFFICIENT MODEL-----------------------
#     # Training model
# #     Efficientnet_model = build_model(base_model_4,
# #                             dropout = dropout,
# #                             fc_layers = FC_LAYERS,
# #                             num_classes = 3)

# #     # compiling the model
# #     opt = tf.keras.optimizers.legacy.RMSprop(learning_rate = 0.0001,decay=1e-4)
# #     Efficientnet_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
# #     # fitting the model
# #     history_efficientnet = Efficientnet_model.fit_generator(training_set,
# #                              steps_per_epoch = len(training_set),
# #                              epochs = 10,
# #                              validation_data = validation_set,
# #                              validation_steps = len(validation_set),
# #                              verbose = 1)

# #     # Saving model
# #     Efficientnet_model.save('/kaggle/working/Efficientnet_'+str(i))

#     shutil.rmtree("/kaggle/working/Image_set/")   ## To delete all directories present in output
#     i+=1

### Visualizing the distribution of different classes in Train, validation and test set

In [29]:
# train_csv_valid['Class'].value_counts(dropna = False).plot(kind = 'bar',grid = True)
# plt.title("Distribtuion of class counts in validation set(after spliting)")
# plt.xticks(rotation = 0)

In [30]:
# train_csv_train['Class'].value_counts(dropna = False).plot(kind = 'bar',grid = True)
# plt.title("Distribtuion of class counts in training set(after spliting)")
# plt.xticks(rotation = 0)

In [31]:
# batch_size=32
# # To handle image loading problem
# from PIL import Image, ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True
# # train_datagen = ImageDataGenerator(rescale = 1./255,
# #                                    shear_range = 0.2,
# #                                    zoom_range = 0.2,
# #                                    horizontal_flip = True, 
# #                                   rotation_range = 30, fill_mode = 'nearest',
# #                                   width_shift_range = 0.2, height_shift_range=0.2,
# #                                   brightness_range = [0.4, 1.5]
# #                                   )
# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True, 
#                                   rotation_range = 10, fill_mode = 'nearest',
#                                   width_shift_range = 0.2, height_shift_range=0.2)
# val_datagen = ImageDataGenerator(rescale = 1./255)
# training_set = train_datagen.flow_from_directory('/kaggle/working/Train/',
#                                    target_size = (224,224),
#                                    batch_size = batch_size,
#                                    class_mode = 'categorical',
#                                     color_mode='rgb')
# validation_set = val_datagen.flow_from_directory('/kaggle/working/Valid/',
#                                   target_size = (224, 224),
#                                   batch_size = batch_size,
#                                   class_mode = 'categorical',
#                                     color_mode='rgb')
# print(training_set.class_indices)

In [32]:
# (_,_)
# training_set[0][0][1]

In [33]:
# train_datagen

# **Basic CNN Model**

In [34]:
# model = Sequential()
# model.add(Conv2D(64, (3, 3), input_shape = (224, 224, 3)))
# model.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3)))
# model.add(LeakyReLU(alpha=0.3))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2, 2)))
# # Dropout
# model.add(Dropout(0.4))
# #Max Pooling
# model.add(Conv2D(32, (3, 3)))

# model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(LeakyReLU(alpha=0.3))
# model.add(BatchNormalization())
# # Dropout
# model.add(Dropout(0.3))
# #Flatten
# model.add(Flatten())
# model.add(Dense(128))
# model.add(LeakyReLU(alpha=0.3))
# model.add(Dense(64))
# model.add(Dropout(0.5))
# model.add(Dense(3, activation = 'softmax'))

In [35]:
# # compiling the model
# model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# #fitting model
# history = model.fit_generator(training_set,
#                          steps_per_epoch = len(training_set),
#                          epochs = 20,
#                          validation_data = validation_set,
#                          validation_steps = len(validation_set),
#                          verbose=1)

#  **ResNet 50**

In [36]:
# # loading resnet model
# Rsnt_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Rsnt_model.trainable = True #--> To freeze all layers excluding top layer

In [37]:
# ## Case 1
# # x = Flatten()(Rsnt_model.output)
# # x = Dense(256, activation='relu')(x)
# # predictions = Dense(3, activation = 'softmax')(x)
# # tl_model = Model(inputs = Rsnt_model.input, outputs = predictions)

# ## Case 2
# # av1 = GlobalAveragePooling2D()(Rsnt_model.output)
# # fc1 = Dense(256, activation='relu')(av1)
# # drp1=Dropout(0.35)(fc1)
# # fc2 = Dense(128, activation='relu')(drp1)
# # drp2=Dropout(0.4)(fc2)
# # bat_norm=BatchNormalization()(drp2)
# # fc3 = Dense(68, activation='relu')(bat_norm)
# # drp3=Dropout(0.25)(fc3)
# # fc4 = Dense(34, activation='relu')(drp3)
# # out = Dense(3, activation='softmax')(fc4)
# # tl_model = Model(inputs=Rsnt_model.input,outputs=out)
# # tl_model.summary()

# ## Case 3
# Rsnt_model = Sequential()

# # 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# # NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
# Rsnt_model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet',input_shape=(224,224,3)))
# Rsnt_model.add(Dense(256, activation = 'relu'))
# Rsnt_model.add(Dropout(0.4))
# Rsnt_model.add(Dense(128, activation = 'relu'))
# Rsnt_model.add(Dropout(0.4))
# Rsnt_model.add(Dense(30, activation = 'relu'))
# Rsnt_model.add(Dropout(0.4))
# # 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
# Rsnt_model.add(Dense(3, activation = 'softmax'))
# # Say not to train first layer (ResNet) model as it is already trained
# Rsnt_model.layers[0].trainable = False

In [38]:
FC_LAYERS = [1024, 512, 256]
dropout = 0.5
def build_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        print(fc)
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)
    preditions = Dense(num_classes, activation='softmax')(x)
    finetune_model = Model(inputs = base_model.input, outputs = preditions)
    return finetune_model

base_model_1 = ResNet50(weights = 'imagenet',
                       include_top = False,
                       input_shape = (224, 224,3))

base_model_2 = InceptionV3(weights = 'imagenet',
                       include_top = False,
                       input_shape = (224,224, 3))

base_model_3 = DenseNet121(weights = 'imagenet',
                       include_top = False,
                       input_shape = (224,224, 3))

base_model_4 = EfficientNetB7(include_top= False,
                              weights="imagenet",
                              input_shape=(224,224,3))

258076736/258076736 [==============================] - 2s 0us/step


### ResNet50 training and saving

In [39]:
# Training model
Rsnt_model = build_model(base_model_1,
                        dropout = dropout,
                        fc_layers = FC_LAYERS,
                        num_classes = 3)

# compiling the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.0001,decay=1e-4)
Rsnt_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
# fitting the model
# history_rsnt = Rsnt_model.fit_generator(training_set,
#                          steps_per_epoch = len(training_set),
#                          epochs = 10,
#                          validation_data = validation_set,
#                          validation_steps = len(validation_set),
#                          verbose = 1)

# Saving model
# Rsnt_model.save('/kaggle/working/Rsnt')

1024
512
256


### InceptionV3 training and saving

In [40]:
# Training model
Inception_model = build_model(base_model_2,
                        dropout = dropout,
                        fc_layers = FC_LAYERS,
                        num_classes = 3)

# compiling the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.0001,decay=1e-4)
Inception_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
# # fitting the model
# history_inception = Inception_model.fit_generator(training_set,
#                          steps_per_epoch = len(training_set),
#                          epochs = 3,
#                          validation_data = validation_set,
#                          validation_steps = len(validation_set),
#                          verbose = 1)

# # Saving model
# Inception_model.save('/kaggle/working/Inception')

1024
512
256


### DenseNet121 training and saving

In [41]:
# Training model
Densenet_model = build_model(base_model_3,
                        dropout = dropout,
                        fc_layers = FC_LAYERS,
                        num_classes = 3)

# compiling the model
opt = tf.keras.optimizers.legacy.RMSprop(learning_rate = 0.0001,decay=1e-4)
Densenet_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
# fitting the model
# history_densenet = Densenet_model.fit_generator(training_set,
#                          steps_per_epoch = len(training_set),
#                          epochs = 10,
#                          validation_data = validation_set,
#                          validation_steps = len(validation_set),
#                          verbose = 1)

# # Saving model
# Densenet_model.save('/kaggle/working/Densenet')

1024
512
256


### EfficientNet B2 training and saving

In [42]:
# Training model
Efficientnet_model = build_model(base_model_4,
                        dropout = dropout,
                        fc_layers = FC_LAYERS,
                        num_classes = 3)

# compiling the model
opt = tf.keras.optimizers.legacy.RMSprop(learning_rate = 0.0001,decay=1e-4)
Efficientnet_model.compile(loss = 'categorical_crossentropy', optimizer= opt, metrics=['accuracy'])
# fitting the model
# history_efficientnet = Efficientnet_model.fit_generator(training_set,
#                          steps_per_epoch = len(training_set),
#                          epochs = 10,
#                          validation_data = validation_set,
#                          validation_steps = len(validation_set),
#                          verbose = 1)

# # Saving model
# Efficientnet_model.save('/kaggle/working/Efficientnet')

1024
512
256


In [43]:
# predic = pd.DataFrame(np.argmax(dense_model.predict(validation_set), axis = 1))

In [44]:
# predic.head()

In [45]:
# score_valid_dense = accuracy_score(predic, train_csv_valid['Class'])
# print(score_valid_dense)

In [46]:
# !zip -r file.zip '/kaggle/working/'

In [47]:
# # Saving model
# Rsnt_model.save('/kaggle/working/my_model')

In [48]:
# Loading model
# new_model = tf.keras.models.load_model('/kaggle/working/my_model')

In [49]:
## Case1: Validation accuracy on ResNet50 model for 30 epochs => 0.7357 --> Image agmentation(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
## Case2: Validation accuracy on ResNet50 model for 30 epochs => 0.7204 --> Image agmentation(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, rotation_range = 30, fill_mode = 'nearest',width_shift_range = 0.2, height_shift_range=0.2,brightness_range = [0.4, 1.5])
## Validation accuracy on ResNet50 model for 20 epochs, loss = categorical => 0.6987 --> Image agmentation(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, rotation_range = 30, fill_mode = 'nearest',width_shift_range = 0.2, height_shift_range=0.2,brightness_range = [0.4, 1.5])
## Validation accuracy on ResNet50 model for 20 epochs, loss = sparse => 0.6331 --> Image agmentation(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, rotation_range = 30, fill_mode = 'nearest',width_shift_range = 0.2, height_shift_range=0.2,brightness_range = [0.4, 1.5])
## Validation accuracy on ResNet50 model for 30 epochs, loss = sparse => 0.6500 --> Image agmentation(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, rotation_range = 30, fill_mode = 'nearest',width_shift_range = 0.2, height_shift_range=0.2,brightness_range = [0.4, 1.5])
# -------RESIZED IMAGES--------
## Case1: Validation accuracy on ResNet50 model for 20 epochs => 0.543 and sparse
## Case2: Validation accuracy on ResNet50 model for 20 epochs => 0.5453 with sigmoid activation fn and sparse
## Case2: Validation accuracy on ResNet50 model for 20 epochs => 0.5453 with relu activation fn and categorical_crossentropy
## Inception v3: RMSprop ,accuracy: 0.65
## Inception v3: Adam ,accuracy: 0.663
## Densenet121: Adam, accuracy: 0.6622
## Densenet121: RMSProp, accuracy: 0.6764
## EfficientNet B2: RMSProp, accuracy: 0.5374
## EfficientNet B2: Adam, accuracy: 0.5374
## EfficientNet B7: RMSProp, accuracy: 0.5374
## EfficientNet B7: Adam, accuracy: 

In [50]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [51]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [52]:
# test_datagen = ImageDataGenerator(rescale = 1./255)
# test_set = test_datagen.flow_from_directory('/kaggle/working/Test/',
#                                   target_size = (224, 224),
#                                   batch_size = batch_size,
#                                   class_mode = 'categorical',
#                                     color_mode='rgb')

In [53]:
# import tensorflow 
# # from tensorflow.keras.models import load_model
# tl_model = tensorflow.keras.models.load_model('/kaggle/input/model-and-its-weights-for-age-detection-problem/age_detection.h5')

### Densenet's prediction 


In [54]:
# dense_model = tf.keras.models.load_model('/kaggle/input/ensembling-age-detection-analytics-vidhya-25-03-23/kaggle_working/Densenet')
# predictions_dense = dense_model.predict(test_set)    
# pred_labels_dense = np.argmax(predictions_dense, axis = 1)
# pred_labels_dense = pd.DataFrame(pred_labels_dense)
# pred_labels_dense.head()

### Efficientnet's prediction

In [55]:
# effic_model = tf.keras.models.load_model('/kaggle/input/ensembling-age-detection-analytics-vidhya-25-03-23/kaggle_working/Efficientnet')
# predictions_effic = effic_model.predict(test_set)    
# pred_labels_effic = np.argmax(predictions_effic, axis = 1)
# pred_labels_effic = pd.DataFrame(pred_labels_effic)
# pred_labels_effic.head()

### Inception's prediction

In [56]:
# inception_model = tf.keras.models.load_model('/kaggle/input/ensembling-age-detection-analytics-vidhya-25-03-23/kaggle_working/Inception')
# predictions_inception = inception_model.predict(test_set)    
# pred_labels_inception = np.argmax(predictions_inception, axis = 1)
# pred_labels_inception = pd.DataFrame(pred_labels_inception)
# pred_labels_inception.head()

### Rsnt's prediction

In [57]:
# rsnt_model = tf.keras.models.load_model('/kaggle/input/ensembling-age-detection-analytics-vidhya-25-03-23/kaggle_working/Rsnt')
# predictions_rsnt = rsnt_model.predict(test_set)    
# pred_labels_rsnt = np.argmax(predictions_rsnt, axis = 1)
# pred_labels_rsnt = pd.DataFrame(pred_labels_rsnt)
# pred_labels_rsnt.head()

In [58]:
## Prediction for test set 
# df = pred_labels_dense.copy()
# df['inception'] = pred_labels_inception
# df['effic'] = pred_labels_effic
# df['rsnt'] = pred_labels_rsnt
# df_final = pd.DataFrame(df)
# df_final.head()
# # df_f = pd.DataFrame(df, columns = ['dense', 'inception', 'effic', 'rsnt'])
# # df_f.head()
# df_1 = df_final.mode(axis = 'columns')
# df_1 = pd.DataFrame(df_1[0])
# df_1.head()

In [59]:
dense_model_1 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet1')
dense_model_2 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet2')
dense_model_3 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet3')
dense_model_4 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet4')
dense_model_5 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet5')

inception_model_1 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Inception_1')
inception_model_2 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Inception_2')
inception_model_3 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Inception_3')
inception_model_4 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Inception_4')
inception_model_5 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Inception_5')

rsnt_model_1 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Rsnt_1')
rsnt_model_2 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Rsnt_2')
rsnt_model_3 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Rsnt_3')
rsnt_model_4 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Rsnt_4')
rsnt_model_5 = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Rsnt_5')

In [60]:
# dm = tf.keras.models.load_model('/kaggle/input/ensembling-models-age-detection-31-03-23/Densenet1')

In [61]:
# dense_model_1.summary()

In [62]:
# Train and validation split using Hold out method
train_csv_valid = train_csv.sample(frac=0.33)
train_csv_train = train_csv.drop(train_csv_valid.index)

# for training dataset
for index, row in tqdm(train_csv_train.iterrows(),total=len(train_csv_train)):
        image_train = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
        image_train = cv2.resize(image_train,(224,224))
        img_path_train = "/kaggle/working/Image_set/Train/"+str(row['Class'])+"/"+row['ID']
        save_img(img_path_train,image_train)
    ## for validation dataset
for index, row in tqdm(train_csv_valid.iterrows(),total=len(train_csv_valid)):
        image_valid = cv2.imread('/kaggle/input/age-detection-analytics-vidhya-peter/train_DETg9GD/Train/'+row['ID'])
        image_valid = cv2.resize(image_valid,(224,224))
        img_path_valid = '/kaggle/working/Image_set/Valid/'+str(row['Class'])+"/"+row['ID']
        save_img(img_path_valid,image_valid)

        
batch_size = 32
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   horizontal_flip = True, 
                                  rotation_range = 10, fill_mode = 'nearest',
                                  width_shift_range = 0.2, height_shift_range=0.2)
val_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/kaggle/working/Image_set/Train/',
                                   target_size = (224,224),
                                   batch_size = batch_size,
                                   class_mode = 'categorical',
                                    color_mode='rgb')
validation_set = val_datagen.flow_from_directory('/kaggle/working/Image_set/Valid/',
                                  target_size = (224, 224),
                                  batch_size = batch_size,
                                  class_mode = 'categorical',
                                    color_mode='rgb')

100%|██████████| 6569/6569 [00:57<00:00, 114.80it/s]


Found 13337 images belonging to 3 classes.
Found 6569 images belonging to 3 classes.


### Max Voting Ensembling

In [63]:
# dm.predict(validation_set)

In [64]:
# predictions_dense1_valid = pd.DataFrame(np.argmax(dm.predict(validation_set), axis = 1))
# predictions_dense1_valid

In [65]:
predictions_dense1_valid = pd.DataFrame(np.argmax(dense_model_1.predict(validation_set), axis = 1))
predictions_dense2_valid = pd.DataFrame(np.argmax(dense_model_2.predict(validation_set), axis = 1))
predictions_dense3_valid = pd.DataFrame(np.argmax(dense_model_3.predict(validation_set), axis = 1))
predictions_dense4_valid = pd.DataFrame(np.argmax(dense_model_4.predict(validation_set), axis = 1))
predictions_dense5_valid = pd.DataFrame(np.argmax(dense_model_5.predict(validation_set), axis = 1))

predictions_inception1_valid = pd.DataFrame(np.argmax(inception_model_1.predict(validation_set), axis = 1))
predictions_inception2_valid = pd.DataFrame(np.argmax(inception_model_2.predict(validation_set), axis = 1))
predictions_inception3_valid = pd.DataFrame(np.argmax(inception_model_3.predict(validation_set), axis = 1))
predictions_inception4_valid = pd.DataFrame(np.argmax(inception_model_4.predict(validation_set), axis = 1))
predictions_inception5_valid = pd.DataFrame(np.argmax(inception_model_5.predict(validation_set), axis = 1))

predictions_rsnt1_valid = pd.DataFrame(np.argmax(rsnt_model_1.predict(validation_set), axis = 1))
predictions_rsnt2_valid = pd.DataFrame(np.argmax(rsnt_model_2.predict(validation_set), axis = 1))
predictions_rsnt3_valid = pd.DataFrame(np.argmax(rsnt_model_3.predict(validation_set), axis = 1))
predictions_rsnt4_valid = pd.DataFrame(np.argmax(rsnt_model_4.predict(validation_set), axis = 1))
predictions_rsnt5_valid = pd.DataFrame(np.argmax(rsnt_model_5.predict(validation_set), axis = 1))

206/206 [==============================] - 16s 74ms/step


In [66]:
# df_valid_new = pd.DataFrame([])
# df_valid_new['dense1'] = predictions_dense4_valid
# df_valid_new.head()

In [67]:
# from sklearn.metrics import accuracy_score
# score_new = accuracy_score(df_valid_new, train_csv_valid['Class'])
# print(score_new)

In [68]:
# predictions_dense_valid.head()

In [69]:
# from sklearn.metrics import accuracy_score
# score3 = accuracy_score(predictions_dense_valid, train_csv_valid['Class'])
# print(score3)

In [70]:
# from sklearn.metrics import accuracy_score
# score4 = accuracy_score(predictions_effic_valid, train_csv_valid['Class'])
# print(score4)

In [71]:
# from sklearn.metrics import accuracy_score
# score5 = accuracy_score(predictions_inception_valid, train_csv_valid['Class'])
# print(score5)

In [72]:
# from sklearn.metrics import accuracy_score
# score6 = accuracy_score(predictions_rsnt_valid, train_csv_valid['Class'])
# print(score6)

In [73]:
# dense_model.predict(validation_set)

In [74]:
# predictions_dense_valid

In [75]:
df_valid = pd.DataFrame([])
df_valid['dense1'] = predictions_dense1_valid
df_valid['dense2'] = predictions_dense2_valid
df_valid['dense3'] = predictions_dense3_valid
df_valid['dense4'] = predictions_dense4_valid
df_valid['dense5'] = predictions_dense5_valid

# df_valid['effic'] = predictions_effic_valid

df_valid['inception1'] = predictions_inception1_valid
df_valid['inception2'] = predictions_inception2_valid
df_valid['inception3'] = predictions_inception3_valid
df_valid['inception4'] = predictions_inception4_valid
df_valid['inception5'] = predictions_inception5_valid

df_valid['rsnt1'] = predictions_rsnt1_valid
df_valid['rsnt2'] = predictions_rsnt2_valid
df_valid['rsnt3'] = predictions_rsnt3_valid
df_valid['rsnt4'] = predictions_rsnt4_valid
df_valid['rsnt5'] = predictions_rsnt5_valid

df_valid.head()

,dense1,dense2,dense3,dense4,dense5,inception1,inception2,inception3,inception4,inception5,rsnt1,rsnt2,rsnt3,rsnt4,rsnt5
0,0,0,0,0,0,0,1,2,1,0,0,2,0,2,2
1,0,0,0,0,2,0,1,2,0,0,0,2,0,0,2
2,2,0,0,0,2,2,2,0,1,0,0,2,0,0,2
3,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0
4,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0


In [76]:
df_temp = df_valid.mode(axis = 'columns')
df_temp.head()

,0,1,2
0,0.0,NaN,NaN
1,0.0,NaN,NaN
2,0.0,NaN,NaN
3,0.0,NaN,NaN
4,0.0,NaN,NaN


In [77]:
df_temp = df_temp[0]
df_temp.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: 0, dtype: float64

In [78]:
df_vv = pd.DataFrame(df_temp)
df_vv.head(25)

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,2.0
8,0.0
9,0.0


In [79]:
df_vv.value_counts()

0.0    5679
2.0     886
1.0       4
dtype: int64

In [80]:
df_vv[0].unique()

array([0., 2., 1.])

In [81]:
train_csv_valid.head()

,ID,Class
15394,9384.jpg,0
17014,23865.jpg,0
19664,12855.jpg,0
18668,7151.jpg,0
16251,15155.jpg,2


In [82]:
from sklearn.metrics import accuracy_score
score = accuracy_score(df_vv, train_csv_valid['Class'])
print(score)

0.5232151012330644


In [83]:
# Stratified K-fold max voting ensembling: 0.544
# Stratified K-fold ensemble averaging: 0.538

# Stratified K-fold max voting ensembling: 0.506
# Stratified K-fold ensemble averaging: 0.5022

# Stratified K-fold max voting ensembling: 0.517
# Stratified K-fold ensemble averaging: 0.517

### Ensemble Averaging 

In [84]:
pr_dense1 = dense_model_1.predict(validation_set)
df_pr_dense1 = pd.DataFrame(pr_dense1)
# df_pr_dense1.head()
pr_dense2 = dense_model_2.predict(validation_set)
df_pr_dense2 = pd.DataFrame(pr_dense2)

pr_dense3 = dense_model_3.predict(validation_set)
df_pr_dense3 = pd.DataFrame(pr_dense3)

pr_dense4 = dense_model_4.predict(validation_set)
df_pr_dense4 = pd.DataFrame(pr_dense4)

pr_dense5 = dense_model_5.predict(validation_set)
df_pr_dense5 = pd.DataFrame(pr_dense5)

206/206 [==============================] - 18s 85ms/step


In [85]:
# pr_effic = effic_model.predict(validation_set)
# df_pr_effic = pd.DataFrame(pr_effic)
# df_pr_effic.head()

In [86]:
pr_inception1 = inception_model_1.predict(validation_set)
df_pr_inception1 = pd.DataFrame(pr_inception1)
# df_pr_inception.head()
pr_inception2 = inception_model_2.predict(validation_set)
df_pr_inception2 = pd.DataFrame(pr_inception2)

pr_inception3 = inception_model_3.predict(validation_set)
df_pr_inception3 = pd.DataFrame(pr_inception3)

pr_inception4 = inception_model_4.predict(validation_set)
df_pr_inception4 = pd.DataFrame(pr_inception4)

pr_inception5 = inception_model_5.predict(validation_set)
df_pr_inception5 = pd.DataFrame(pr_inception5)

206/206 [==============================] - 14s 68ms/step


In [87]:
pr_rsnt1 = rsnt_model_1.predict(validation_set)
df_pr_rsnt1 = pd.DataFrame(pr_rsnt1)
# df_pr_rsnt.head()
pr_rsnt2 = rsnt_model_2.predict(validation_set)
df_pr_rsnt2 = pd.DataFrame(pr_rsnt2)

pr_rsnt3 = rsnt_model_3.predict(validation_set)
df_pr_rsnt3 = pd.DataFrame(pr_rsnt3)

pr_rsnt4 = rsnt_model_4.predict(validation_set)
df_pr_rsnt4 = pd.DataFrame(pr_rsnt4)

pr_rsnt5 = rsnt_model_5.predict(validation_set)
df_pr_rsnt5 = pd.DataFrame(pr_rsnt5)

206/206 [==============================] - 15s 71ms/step


In [88]:
df_0 = pd.DataFrame()
df_0['dense_0_1'] = df_pr_dense1[0]
df_0['dense_0_2'] = df_pr_dense2[0]
df_0['dense_0_3'] = df_pr_dense3[0]
df_0['dense_0_4'] = df_pr_dense4[0]
df_0['dense_0_5'] = df_pr_dense5[0]
# df_0['effic_0'] = df_pr_effic[0]

df_0['inception_0_1'] = df_pr_inception1[0]
df_0['inception_0_2'] = df_pr_inception2[0]
df_0['inception_0_3'] = df_pr_inception3[0]
df_0['inception_0_4'] = df_pr_inception4[0]
df_0['inception_0_5'] = df_pr_inception5[0]

df_0['rsnt_0_1'] = df_pr_rsnt1[0]
df_0['rsnt_0_2'] = df_pr_rsnt2[0]
df_0['rsnt_0_3'] = df_pr_rsnt3[0]
df_0['rsnt_0_4'] = df_pr_rsnt4[0]
df_0['rsnt_0_5'] = df_pr_rsnt5[0]

df_0['mean'] = df_0.mean(axis = 1)
df_0.head()

,dense_0_1,dense_0_2,dense_0_3,dense_0_4,dense_0_5,inception_0_1,inception_0_2,inception_0_3,inception_0_4,inception_0_5,rsnt_0_1,rsnt_0_2,rsnt_0_3,rsnt_0_4,rsnt_0_5,mean
0,0.979371,0.530836,0.951885,1.000000,0.000091,0.021257,0.113036,0.191226,0.939342,0.987652,0.999695,0.750822,0.949977,0.999966,0.995552,0.694047
1,1.000000,1.000000,0.909598,0.635319,0.833679,0.999695,0.000211,0.999975,0.130379,0.999844,0.997968,0.083821,0.998042,0.922725,0.981492,0.766183
2,0.979839,0.002949,0.679365,0.968173,0.995512,0.866677,0.813809,0.849490,0.031365,0.110524,0.998118,0.015384,0.953505,0.982528,0.009787,0.617135
3,0.971502,0.003731,0.973913,0.463900,0.484981,0.982328,0.999857,0.854557,0.004659,0.965158,0.999942,0.816914,0.999789,0.787538,0.000232,0.687267
4,0.393598,0.818612,0.000014,0.995704,1.000000,0.965046,0.897951,0.000276,0.999464,0.024874,0.000019,0.000544,0.001180,0.658464,0.005378,0.450742


In [89]:
df_1 = pd.DataFrame()
df_1['dense_1_1'] = df_pr_dense1[1]
df_1['dense_1_2'] = df_pr_dense2[1]
df_1['dense_1_3'] = df_pr_dense3[1]
df_1['dense_1_4'] = df_pr_dense4[1]
df_1['dense_1_5'] = df_pr_dense5[1]
# df_0['effic_0'] = df_pr_effic[0]

df_1['inception_1_1'] = df_pr_inception1[1]
df_1['inception_1_2'] = df_pr_inception2[1]
df_1['inception_1_3'] = df_pr_inception3[1]
df_1['inception_1_4'] = df_pr_inception4[1]
df_1['inception_1_5'] = df_pr_inception5[1]

df_1['rsnt_1_1'] = df_pr_rsnt1[1]
df_1['rsnt_1_2'] = df_pr_rsnt2[1]
df_1['rsnt_1_3'] = df_pr_rsnt3[1]
df_1['rsnt_1_4'] = df_pr_rsnt4[1]
df_1['rsnt_1_5'] = df_pr_rsnt5[1]

df_1['mean'] = df_1.mean(axis = 1)
df_1.head()

,dense_1_1,dense_1_2,dense_1_3,dense_1_4,dense_1_5,inception_1_1,inception_1_2,inception_1_3,inception_1_4,inception_1_5,rsnt_1_1,rsnt_1_2,rsnt_1_3,rsnt_1_4,rsnt_1_5,mean
0,0.007660,1.831592e-01,5.475913e-03,7.210887e-18,1.395563e-09,0.000812,1.710898e-03,6.520163e-01,0.001943,0.009186,2.741259e-04,0.074903,4.329119e-03,0.000006,1.512648e-04,0.062775
1,0.000000,0.000000e+00,4.104149e-02,1.888564e-01,6.852714e-02,0.000242,6.692578e-09,8.955997e-07,0.010826,0.000084,6.516960e-05,0.000401,1.015938e-03,0.053475,2.845262e-03,0.024492
2,0.013214,1.472588e-07,2.178137e-01,1.467238e-02,5.533468e-04,0.002935,1.852457e-01,5.950131e-02,0.966269,0.885549,1.580252e-03,0.000157,1.449056e-02,0.000657,2.449377e-05,0.157511
3,0.014269,3.303362e-07,2.058949e-02,2.965990e-03,3.188119e-01,0.016291,2.900151e-06,5.367160e-02,0.000044,0.008266,3.765305e-06,0.058702,8.645965e-05,0.140501,2.320489e-09,0.042280
4,0.026536,1.904565e-02,3.104315e-11,4.024756e-03,3.021430e-11,0.017831,8.547399e-04,1.982998e-07,0.000017,0.000147,2.922979e-10,0.999446,2.140839e-08,0.176184,3.358542e-06,0.082939


In [90]:
df_2 = pd.DataFrame()
df_2['dense_2_1'] = df_pr_dense1[2]
df_2['dense_2_2'] = df_pr_dense2[2]
df_2['dense_2_3'] = df_pr_dense3[2]
df_2['dense_2_4'] = df_pr_dense4[2]
df_2['dense_2_5'] = df_pr_dense5[2]
# df_0['effic_0'] = df_pr_effic[0]

df_2['inception_2_1'] = df_pr_inception1[2]
df_2['inception_2_2'] = df_pr_inception2[2]
df_2['inception_2_3'] = df_pr_inception3[2]
df_2['inception_2_4'] = df_pr_inception4[2]
df_2['inception_2_5'] = df_pr_inception5[2]

df_2['rsnt_2_1'] = df_pr_rsnt1[2]
df_2['rsnt_2_2'] = df_pr_rsnt2[2]
df_2['rsnt_2_3'] = df_pr_rsnt3[2]
df_2['rsnt_2_4'] = df_pr_rsnt4[2]
df_2['rsnt_2_5'] = df_pr_rsnt5[2]

df_2['mean'] = df_2.mean(axis = 1)
df_2.head()

,dense_2_1,dense_2_2,dense_2_3,dense_2_4,dense_2_5,inception_2_1,inception_2_2,inception_2_3,inception_2_4,inception_2_5,rsnt_2_1,rsnt_2_2,rsnt_2_3,rsnt_2_4,rsnt_2_5,mean
0,0.012968,0.286005,0.042639,2.927835e-14,9.999093e-01,0.977930,0.885254,0.156758,0.058715,0.003162,0.000031,0.174275,0.045693,0.000028,0.004297,0.243178
1,0.000000,0.000000,0.049361,1.758242e-01,9.779395e-02,0.000062,0.999789,0.000024,0.858795,0.000072,0.001967,0.915778,0.000942,0.023800,0.015662,0.209325
2,0.006947,0.997051,0.102822,1.715449e-02,3.934662e-03,0.130388,0.000945,0.091009,0.002366,0.003927,0.000302,0.984459,0.032004,0.016815,0.990188,0.225354
3,0.014229,0.996269,0.005497,5.331338e-01,1.962069e-01,0.001380,0.000140,0.091771,0.995297,0.026576,0.000054,0.124384,0.000125,0.071961,0.999768,0.270453
4,0.579866,0.162343,0.999986,2.716996e-04,3.537598e-09,0.017123,0.101195,0.999724,0.000519,0.974979,0.999980,0.000009,0.998820,0.165352,0.994619,0.466319


In [91]:
df_mean = pd.DataFrame()
df_mean['mean_0'] = df_0['mean']
df_mean['mean_1'] = df_1['mean']
df_mean['mean_2'] = df_2['mean']
df_mean['max'] = df_mean.max(axis = 'columns')
df_mean.head()

,mean_0,mean_1,mean_2,max
0,0.694047,0.062775,0.243178,0.694047
1,0.766183,0.024492,0.209325,0.766183
2,0.617135,0.157511,0.225354,0.617135
3,0.687267,0.042280,0.270453,0.687267
4,0.450742,0.082939,0.466319,0.466319


In [92]:
# df_mean.head()

In [93]:
# df_t = df_mean.drop(['max'],inplace = True)
# df_t.head()

In [94]:
df_class = pd.DataFrame(np.argmax(np.array(df_mean), axis = 1))
df_class.head()

,0
0,0
1,0
2,0
3,0
4,2


In [95]:
df_class.head(20)

,0
0,0
1,0
2,0
3,0
4,2
5,2
6,0
7,0
8,0
9,0


In [96]:
from sklearn.metrics import accuracy_score
score2 = accuracy_score(df_class, train_csv_valid['Class'])
print(score2)

0.5186481960724616


In [97]:
# df_mean.head()

In [98]:
# np.array(df_mean)

In [99]:
# predictions = new_model.predict(test_set)    
# pred_labels = np.argmax(predictions, axis = 1)
# pred_labels.head()

In [100]:
# pred_labels = pd.DataFrame(pred_labels, columns = ['Class'])
# pred_labels.head()

In [101]:
# pred_labels = pred_labels.replace([0,1,2], ['MIDDLE', 'OLD', 'YOUNG'])
# pred_labels.head()

In [102]:
# import os 
# dir_list = os.listdir('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/Test/')
# # print(dir_list) --> To print the list of the files
# dir_list = pd.DataFrame(dir_list, columns = ['ID'])
# dir_list.sort_values(by = 'ID', inplace = True)
# dir_list.head()

In [103]:
# dir_list = dir_list.reset_index(drop = True)
# # dir_list.drop(['index'], axis = 1)
# dir_list.head()

In [104]:
# test_final = pd.concat([dir_list, pred_labels], axis = 1)
# test_final.head(20)

In [105]:
# test_df = pd.read_csv('/kaggle/input/age-detection-analytics-vidhya-peter/test_Bh8pGW3/test.csv')
# test_df.head()

In [106]:
# test_df_final = test_df.merge(test_final, how = 'left')
# test_df_final.head(20)

In [107]:
# test_df_final.unique()

In [108]:
# test_df_final = test_df_final.reindex(columns = ['Class','ID'])
# test_df_final.head()

In [109]:
# test_df_final.to_csv('submission.csv', index = False)

In [110]:
# submission = pd.read_csv('/kaggle/working/submission.csv')
# submission.head()

In [111]:
# ## Saving model
# tl_model.save('/kaggle/working/age_detection.h5')
# print('Model Saved!')

In [112]:
# ## Saving weights
# tl_model.save_weights('/kaggle/working/age_detection_weights.h5')
# print('Weight saved!')